In [90]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import pubmed as pm
import pandas as pd
import time
import os

In [10]:
path = f'{os.getcwd()}/PDBs'

In [3]:
dfs = []
for i in range(1, 6):
    df_aux = pd.read_csv(f'{path}/DBBASP_releases/peptides-{i}.csv')
    dfs.append(df_aux)

In [5]:
df = pd.concat(dfs, axis=0, ignore_index=True)
#df = pd.read_csv(f'{path}/DBBASP_releases/DBBASP_2023_01_18.csv')

In [6]:
#Add link from DBBASP for each peptide
base_url = 'https://dbaasp.org/peptide-card?id=DBAASP'
links = []
for i in range(0, len(df)):
    links.append(f'{base_url}{df.loc[i]["SYNTHESIS TYPE"][:1]}_{df.loc[i]["ID"]}')
df['LINK'] = links

In [101]:
driver = webdriver.Chrome()

In [102]:
# Get DOI from PubMed
doi_list = []

for link in df['LINK']:
    count += 1
    driver.get(link)
    time.sleep(2)
    table = driver.find_element(By.XPATH, '/html/body/main/div/main/div[1]/div[1]/div/div/div[18]/div/div[2]/div/div/div[2]/table/tbody')
    pubmed_links = table.find_elements(By.TAG_NAME, 'a')
    pmids = [pmid.get_attribute('href').split('pubmed/')[1] for pmid in pubmed_links]
    current_doi = []
    for pmid in pmids:
        current_doi.append(pm.get_doi_from_pmid(driver, pmid))
    doi_list.append(current_doi)
    
df['DOI'] = doi_list

In [103]:
driver.close()

In [7]:
dt = datetime.now()
df.to_csv(f'{path}/DBBASP_releases/DBBASP_{dt.strftime("%Y_%m_%d")}.csv', index=False)
df.to_csv(f'{path}/DBBASP.csv', index=False)

In [12]:
df

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT,LINK
0,1,Multimer,Distinctin,NaN,NLVSGLIEARKYLEQLHRKLKNCKV ENREVPPGFTALIKTLR...,NaN,Ribosomal,Gram+ Gram-,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_1
1,3,Multimer,Halocidin,NaN,WLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Ribosomal,Gram+,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_3
2,4,Multimer,Khal,NaN,KWLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Synthetic,Gram+ Gram-,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPS_4
3,6,Multi-Peptide,Enterocin X,NaN,SNDSLWYGVGQFMGKQANCITNHPVKHMIIPGYCLSKILG IA...,NaN,Ribosomal,Gram+,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_6
4,7,Multi-Peptide,EAFP1 + EAFP2,NaN,XTCASRCPRPCNAGLCCSIYGYCGSGNAYCGAGNCRCQCRG X...,NaN,Ribosomal,Fungus,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_7
...,...,...,...,...,...,...,...,...,...,...
19804,20057,Monomer,NaN,NaN,LWWRRRWWL,AMD,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20057
19805,20058,Monomer,NaN,NaN,FWWRRRWWF,AMD,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20058
19806,20059,Monomer,NaN,NaN,WWWRRRWWW,AMD,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20059
19807,20060,Monomer,NaN,NaN,VWWRRRWWV,AMD,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20060
